# Importing libraries

In [295]:
from pyzotero import zotero
import os
import tweepy as tw
import pandas as pd
import datetime
import json, sys
from datetime import date, timedelta  
import datetime
import plotly.express as px
import pycountry
import re
import pandas as pd
import requests
import time
from datetime import datetime, timedelta

In [296]:
"# All items in the Zotero Intelligence bibliography library".index("All")

2

In [297]:
library_id = '2514686'
library_type = 'group'
api_key = '' # api_key is only needed for private groups and libraries
zot = zotero.Zotero(library_id, library_type)


# All items in the Zotero Intelligence bibliography library

In [298]:
items = zot.everything(zot.top())

data3=[]
columns3=['Title','Publication type', 'Link to publication', 'Abstract', 'Zotero link', 'Date published', 'FirstName2', 'Publisher', 'Journal', 'Date added', 'Col key', 'DOI', 'Book_title']

for item in items:
    # Extracting various types of creators
    authors = ""
    if 'creators' in item['data']:
        authors_info = item['data']['creators']
        author_names = []
        for author in authors_info:
            creator_type = author.get('creatorType')
            valid_types = ['author', 'contributor', 'editor', 'guest', 'podcaster', 'presenter', 'translator', 'programmer']
            
            if item['data']['itemType'] == 'bookSection' and creator_type == 'editor':
                continue  # Skip adding 'editor' for Book chapters
            
            if creator_type in valid_types:
                if 'firstName' in author and 'lastName' in author:
                    author_names.append(author['firstName'] + ' ' + author['lastName'])
                elif 'name' in author:
                    author_names.append(author['name'])
                
        authors = ', '.join(author_names)
        
    data3.append((
        item['data']['title'], 
        item['data']['itemType'], 
        item['data']['url'], 
        item['data']['abstractNote'], 
        item['links']['alternate']['href'],
        item['data'].get('date'),
        authors,  # Add the concatenated authors string
        item['data'].get('publisher'),
        item['data'].get('publicationTitle'),
        item['data']['dateAdded'],
        item['data']['collections'],
        item['data'].get('DOI'),
        item['data'].get('bookTitle')
        ))
# pd.set_option('display.max_colwidth', None)

df = pd.DataFrame(data3, columns=columns3)

mapping_types = {
    'thesis': 'Thesis',
    'journalArticle': 'Journal article',
    'book': 'Book',
    'bookSection': 'Book chapter',
    'blogPost': 'Blog post',
    'videoRecording': 'Video',
    'podcast': 'Podcast',
    'magazineArticle': 'Magazine article',
    'webpage': 'Webpage',
    'newspaperArticle': 'Newspaper article',
    'report': 'Report',
    'forumPost': 'Forum post',
    'manuscript': 'Manuscript',
    'document': 'Document',
    'conferencePaper': 'Conference paper',
    'film': 'Film',
    'presentation': 'Presentation',
    'audioRecording':'Podcast',
    'preprint':'Preprint',
    'hearing':'Hearing',
    'computerProgram':'Computer program'
}
df['Publication type'] = df['Publication type'].replace(mapping_types)

mapping_publisher = {
    'Taylor & Francis Group': 'Taylor and Francis',
    'Taylor and Francis': 'Taylor and Francis',
    'Taylor & Francis': 'Taylor and Francis',
    'Routledge': 'Routledge',
    'Routledge Handbooks Online': 'Routledge',
    'Praeger Security International': 'Praeger',
    'Praeger': 'Praeger'
}
df['Publisher'] = df['Publisher'].replace(mapping_publisher)

# df['Publisher'] = df['Publisher'].replace(['Taylor & Francis Group', 'Taylor and Francis', 'Taylor & Francis'], 'Taylor and Francis')
# df['Publisher'] = df['Publisher'].replace(['Routledge', 'Routledge Handbooks Online'], 'Routledge')
# df['Publisher'] = df['Publisher'].replace(['Praeger Security International', 'Praeger'], 'Praeger')

mapping_journal = {
    'International Journal of Intelligence and Counterintelligence': 'Intl Journal of Intelligence and Counterintelligence',
    'International Journal of Intelligence and CounterIntelligence': 'Intl Journal of Intelligence and Counterintelligence',
    'International Journal of Intelligence and Counter Intelligence': 'Intl Journal of Intelligence and Counterintelligence',
    'Intelligence and national security': 'Intelligence and National Security',
    'Intelligence and National Security': 'Intelligence and National Security',
    'Intelligence & National Security': 'Intelligence and National Security'
}

df['Journal'] = df['Journal'].replace(mapping_journal)

# df['Journal'] = df['Journal'].replace(['International Journal of Intelligence and Counterintelligence', 'International Journal of Intelligence and CounterIntelligence'], 'Intl Journal of Intelligence and Counterintelligence')
# df['Journal'] = df['Journal'].replace(['Intelligence and national security', 'Intelligence and National Security', 'Intelligence & National Security'], 'Intelligence and National Security')

In [299]:
df_test = df[df['Title']=='Intelligence Practice and Democratic Oversight']
df_test

,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,Date added,Col key,DOI,Book_title
964,Intelligence Practice and Democratic Oversight,Report,https://www.dcaf.ch/intelligence-practice-and-...,,https://www.zotero.org/groups/intelligence_bib...,2003-07-01,Geneva Centre for Security Sector Governance,None,None,2024-02-19T11:03:09Z,[DVEM4H4W],None,None


In [300]:
# items = zot.everything(zot.top())

# data3=[]
# columns3=['Title','Publication type', 'Link to publication', 'Abstract', 'Zotero link', 'Date published', 'FirstName2', 'Publisher', 'Journal', 'Date added', 'Col key']

# for item in items:
#     # Extracting various types of creators
#     authors = ""
#     if 'creators' in item['data']:
#         authors_info = item['data']['creators']
#         author_names = []
#         for author in authors_info:
#             creator_type = author.get('creatorType')
#             valid_types = ['author', 'contributor', 'editor', 'guest', 'podcaster']
#             if creator_type in valid_types:  # Include all identified creator types
#                 if 'firstName' in author and 'lastName' in author:
#                     author_names.append(author['firstName'] + ' ' + author['lastName'])
#                 elif 'name' in author:
#                     author_names.append(author['name'])  # Handling cases where 'firstName' and 'lastName' are absent
                
#         authors = ', '.join(author_names)
        
#     data3.append((
#         item['data']['title'], 
#         item['data']['itemType'], 
#         item['data']['url'], 
#         item['data']['abstractNote'], 
#         item['links']['alternate']['href'],
#         item['data'].get('date'),
#         authors,  # Add the concatenated authors string
#         item['data'].get('publisher'),
#         item['data'].get('publicationTitle'),
#         item['data']['dateAdded'],
#         item['data']['collections']
#         )) 
# # pd.set_option('display.max_colwidth', None)

# df = pd.DataFrame(data3, columns=columns3)

# mapping_types = {
#     'thesis': 'Thesis',
#     'journalArticle': 'Journal article',
#     'book': 'Book',
#     'bookSection': 'Book chapter',
#     'blogPost': 'Blog post',
#     'videoRecording': 'Video',
#     'podcast': 'Podcast',
#     'magazineArticle': 'Magazine article',
#     'webpage': 'Webpage',
#     'newspaperArticle': 'Newspaper article',
#     'report': 'Report',
#     'forumPost': 'Forum post',
#     'manuscript': 'Manuscript',
#     'document': 'Document',
#     'conferencePaper': 'Conference paper',
#     'film': 'Film',
#     'presentation': 'Presentation',
#     'audioRecording':'Podcast'
# }
# df['Publication type'] = df['Publication type'].replace(mapping_types)

# mapping_publisher = {
#     'Taylor & Francis Group': 'Taylor and Francis',
#     'Taylor and Francis': 'Taylor and Francis',
#     'Taylor & Francis': 'Taylor and Francis',
#     'Routledge': 'Routledge',
#     'Routledge Handbooks Online': 'Routledge',
#     'Praeger Security International': 'Praeger',
#     'Praeger': 'Praeger'
# }
# df['Publisher'] = df['Publisher'].replace(mapping_publisher)

# # df['Publisher'] = df['Publisher'].replace(['Taylor & Francis Group', 'Taylor and Francis', 'Taylor & Francis'], 'Taylor and Francis')
# # df['Publisher'] = df['Publisher'].replace(['Routledge', 'Routledge Handbooks Online'], 'Routledge')
# # df['Publisher'] = df['Publisher'].replace(['Praeger Security International', 'Praeger'], 'Praeger')

# mapping_journal = {
#     'International Journal of Intelligence and Counterintelligence': 'Intl Journal of Intelligence and Counterintelligence',
#     'International Journal of Intelligence and CounterIntelligence': 'Intl Journal of Intelligence and Counterintelligence',
#     'Intelligence and national security': 'Intelligence and National Security',
#     'Intelligence and National Security': 'Intelligence and National Security',
#     'Intelligence & National Security': 'Intelligence and National Security'
# }

# df['Journal'] = df['Journal'].replace(mapping_journal)

# # df['Journal'] = df['Journal'].replace(['International Journal of Intelligence and Counterintelligence', 'International Journal of Intelligence and CounterIntelligence'], 'Intl Journal of Intelligence and Counterintelligence')
# # df['Journal'] = df['Journal'].replace(['Intelligence and national security', 'Intelligence and National Security', 'Intelligence & National Security'], 'Intelligence and National Security')

In [301]:
def zotero_collections2(library_id, library_type):
    collections = zot.collections()
    data = [(item['data']['key'], item['data']['name'], item['meta']['numItems'], item['links']['alternate']['href']) for item in collections]
    df_collections = pd.DataFrame(data, columns=['Key', 'Name', 'Number', 'Link'])
    return df_collections
df_collections_2 = zotero_collections2(library_id, library_type)

In [302]:
def duplicate_rows_by_col_key(df, df_collections):
    # Duplicate rows based on 'Col key'
    duplicated_rows = []
    for index, row in df.iterrows():
        if isinstance(row['Col key'], list):
            for key in row['Col key']:
                new_row = row.copy()
                collection_info = df_collections[df_collections['Key'] == key]
                if not collection_info.empty:
                    new_row['Collection_Key'] = key
                    new_row['Collection_Name'] = collection_info.iloc[0]['Name']
                    new_row['Collection_Link'] = collection_info.iloc[0]['Link']
                    duplicated_rows.append(new_row)
        else:
            key = row['Col key']
            new_row = row.copy()
            collection_info = df_collections[df_collections['Key'] == key]
            if not collection_info.empty:
                new_row['Collection_Key'] = key
                new_row['Collection_Name'] = collection_info.iloc[0]['Name']
                new_row['Collection_Link'] = collection_info.iloc[0]['Link']
                duplicated_rows.append(new_row)

    # Create a new DataFrame with duplicated rows
    duplicated_df = pd.DataFrame(duplicated_rows)

    return duplicated_df

# Duplicating rows based on 'Col key' and collections information
duplicated_df = duplicate_rows_by_col_key(df, df_collections_2)

In [303]:
# day_allitems = datetime.date.today().isoformat()

In [304]:
# df_fa = df['FirstName2']
# df_fa = pd.DataFrame(df_fa.tolist())
# df_fa = df_fa[0]
# df_fa = df_fa.apply(lambda x: {} if pd.isna(x) else x)
# df_new = pd.json_normalize(df_fa, errors='ignore') 
# df = pd.concat([df, df_new], axis=1)
# df['firstName'] = df['firstName'].fillna('null')
# df['lastName'] = df['lastName'].fillna('null')

In [305]:
df.to_csv('all_items.csv')
duplicated_df.to_csv('all_items_duplicated.csv')

In [306]:
import psycopg2
from sqlalchemy import create_engine
import psycopg2.extras as extras

# Countries 

In [307]:
# Read the csv file into a pandas dataframe
df = pd.read_csv('https://raw.githubusercontent.com/YusufAliOzkan/zotero-intelligence-bibliography/main/all_items.csv')

# Dictionary to map non-proper country names to their proper names
country_map = {
    'british': 'UK',
    'great britain': 'UK',
    'UK' : 'UK', 
    'america' : 'United States',
    'United States of America' : 'United States',
    'Soviet Union': 'Russia', 
    'american' : 'United States',
    'United States' : 'United States',
    'russian' : 'Russia'
    # Add more mappings as needed
}

# Find the country names in the "title" column of the dataframe
found_countries = {}
for i, row in df.iterrows():
    title = str(row['Title']).lower()
    for country in pycountry.countries:
        name = country.name.lower()
        if name in title or (name + 's') in title:  # Check for singular and plural forms of country names
            proper_name = country.name
            found_countries[proper_name] = found_countries.get(proper_name, 0) + 1
    for non_proper, proper in country_map.items():
        if non_proper in title:
            found_countries[proper] = found_countries.get(proper, 0) + title.count(non_proper)

# Create a new dataframe containing the found countries and their counts
df_countries = pd.DataFrame({'Country': list(found_countries.keys()), 'Count': list(found_countries.values())})

In [308]:
fig = px.choropleth(df_countries, locations='Country', locationmode='country names', color='Count', 
                    title='Country mentions in titles', color_continuous_scale='Viridis',
                    width=1100, height=700) # Adjust the size of the map here

# Display the map
fig.show()

In [309]:
df_countries=df_countries.sort_values(by='Count', ascending=False)
df_countries.reset_index(drop=True)

,Country,Count
0,United States,685
1,UK,465
2,Israel,150
3,Russia,149
4,Ukraine,123
...,...,...
131,Croatia,1
132,Sierra Leone,1
133,Estonia,1
134,Togo,1


In [310]:
fig = px.choropleth(df_countries, locations='Country', locationmode='country names', color='Count', 
                    title='Country mentions in titles', color_continuous_scale='Viridis',
                    width=1100, height=700) # Adjust the size of the map here

# Display the map
fig.show()

In [311]:
df_countries.to_csv('countries.csv',index=False)

# NER analysis

In [312]:
import spacy
import nltk
import ast
from spacy.pipeline import EntityRecognizer

In [313]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
df = pd.read_csv('https://raw.githubusercontent.com/YusufAliOzkan/zotero-intelligence-bibliography/main/all_items.csv')
nlp = spacy.load("en_core_web_sm")
ruler = nlp.add_pipe("entity_ruler")
patterns = [{"label": "ORG", "pattern": "MI6"}]
ruler.add_patterns(patterns)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\yaozk\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
c:\Users\yaozk\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:865: UserWarning:

[W095] Model 'en_core_web_sm' (3.5.0) was trained with spaCy v3.5 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible

In [314]:
def extract_entities(text):
    doc = nlp(text)
    orgs = []
    gpes = []
    people = []
    for entity in doc.ents:
        if entity.label_ == 'ORG':
            orgs.append(entity.text)
        elif entity.label_ == 'GPE':
            gpes.append(entity.text)
        elif entity.label_ == 'PERSON':
            people.append(entity.text)
    return pd.Series({'ORG': orgs, 'GPE': gpes, 'PERSON': people})

In [315]:
df_title = df[['Title']].copy()
df_title = df_title.rename(columns={'Title':'Text'})
df_abstract = df[['Abstract']].copy()
df_abstract = df_abstract.rename(columns={'Abstract':'Text'})
df_one = pd.concat([df_title, df_abstract], ignore_index=True)
df_one['Text'] = df_one['Text'].fillna('')

In [316]:
df_one = pd.concat([df_one[['Text']], df_one['Text'].apply(extract_entities)], axis=1)
df_one = df_one.explode('GPE').reset_index(drop=True)
df_one = df_one.explode('ORG').reset_index(drop=True)
df_one = df_one.explode('PERSON').reset_index(drop=True)

In [317]:
df_one_g = df_one.copy()
df_one_g = df_one[['Text', 'GPE']]
# df_one_g = df_one_g.fillna('')
df_one_g = df_one_g.drop_duplicates(subset=['Text', 'GPE'])

gpe_counts = df_one_g['GPE'].value_counts().reset_index()
gpe_counts.columns = ['GPE', 'count']

# pd.options.display.max_rows = None


mapping_locations = {
    'the United States': 'USA',
    'The United States': 'USA',
    'US': 'USA',
    'U.S.': 'USA',
    'United States' : 'USA',
    'America' : 'USA',
    'the United States of America' : 'USA',
    'Britain' : 'UK',
    'the United Kingdom': 'UK',
    'U.K.' : 'UK',
    'Global Britain' : 'UK',
    'United Kingdom' : 'UK', 
    'the Soviet Union' : 'Russia',
    'The Soviet Union' : 'Russia',
    'USSR' : 'Russia',
    'Ukraine - Perspective' : 'Ukraine',
    'Ukrainian' : 'Ukraine',
    'Great Britain' : 'UK',
    'Ottoman Empire' : 'Turkey'
}
gpe_counts['GPE'] =gpe_counts['GPE'].replace(mapping_locations)
gpe_counts = gpe_counts.groupby('GPE').sum().reset_index()
gpe_counts.sort_values('count', ascending=False, inplace=True)
gpe_counts = gpe_counts.reset_index(drop=True)
gpe_counts.head(15)

,GPE,count
0,USA,1455
1,UK,753
2,Russia,546
3,Germany,224
4,Ukraine,220
5,Israel,185
6,China,172
7,France,151
8,Canada,148
9,Iraq,135


In [318]:
df_one_p = df_one.copy()
df_one_p = df_one[['Text', 'PERSON']]
# df_one_p = df_one_g.fillna('')
df_one_p = df_one_p.drop_duplicates(subset=['Text', 'PERSON'])

person_counts = df_one_p['PERSON'].value_counts().reset_index()
person_counts.columns = ['PERSON', 'count']

mapping_person = {
    'Putin' : 'Vladimir Putin',
    'Vladimir Putin' : 'Vladimir Putin',
    'Churchill' : 'Winston Churchill',
    'Hitler' : 'Adolf Hitler',
    'Biden' : 'Joe Biden',
    "John le Carré’s" : "John le Carré"
}

person_counts['PERSON'] =person_counts['PERSON'].replace(mapping_person)
person_counts = person_counts.groupby('PERSON').sum().reset_index()
person_counts.sort_values('count', ascending=False, inplace=True)

remove_person = ['MI6', 'Twitter', 'GRU'
          ]
person_counts = person_counts[~person_counts['PERSON'].isin(remove_person)]
person_counts = person_counts.reset_index(drop=True)

person_counts.head(15)

,PERSON,count
0,Adolf Hitler,89
1,Vladimir Putin,78
2,Winston Churchill,56
3,Andrew Hammond,53
4,Stalin,43
5,James Bond,30
6,Eisenhower,29
7,Bush,28
8,Mujibur Rahman,28
9,John le Carré,25


In [319]:
df_one_o = df_one.copy()
df_one_o = df_one[['Text', 'ORG']]
# df_one_p = df_one_g.fillna('')
df_one_o = df_one_o.drop_duplicates(subset=['Text', 'ORG'])

org_counts = df_one_o['ORG'].value_counts().reset_index()
org_counts.columns = ['ORG', 'count']

mapping_organisations = {
    'The British Secret Intelligence Service' : 'SIS',
    'the British Secret Intelligence Service' : 'SIS',
    'The Joint Intelligence Committee' : 'Joint Intelligence Committee',
    'the Joint Intelligence Committee' : 'Joint Intelligence Committee',
    'Joint Intelligence Committee' : 'Joint Intelligence Committee',
    'the Joint Intelligence Committee - History' : 'Joint Intelligence Committee',
    'Central Intelligence Agency' : 'CIA',
    'the Central Intelligence Agency' : 'CIA',
    'the Foreign Office' : 'Foreign Office',
    'Schar School' : 'Schar School of Policy and Government',
    'the Secret Intelligence Service' : 'SIS',
    "George Mason University's" : "George Mason University",
    'JIC' : 'Joint Intelligence Committee'
}
org_counts['ORG'] =org_counts['ORG'].replace(mapping_organisations)
org_counts = org_counts.groupby('ORG').sum().reset_index()
org_counts.sort_values('count', ascending=False, inplace=True)

remove_orgs = ['Intelligence', 'Kremlin', 'Ultra', 'International Security', 'Intelligence Studies', 'Intelligence Analysis', 'OSINT']
org_counts = org_counts[~org_counts['ORG'].isin(remove_orgs)]
org_counts = org_counts.reset_index(drop=True)

org_counts.head(15)

,ORG,count
0,CIA,742
1,KGB,149
2,FBI,108
3,Allied,103
4,Congress,78
5,SIS,69
6,SOE,66
7,the Intelligence Community,65
8,NATO,64
9,EU,56


In [320]:
gpe_counts.head(15).to_csv('gpe.csv')
person_counts.head(15).to_csv('person.csv')
org_counts.head(15).to_csv('org.csv')

In [321]:
test = pd.read_csv('all_items.csv')
test

,Unnamed: 0,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,Date added,Col key,DOI,Book_title
0,0,France : French intelligence services face pos...,Newspaper article,https://www.intelligenceonline.com/government-...,The looming possibility of having a president ...,https://www.zotero.org/groups/intelligence_bib...,2024-06-18T04:00:00Z,Intelligence Online,NaN,Intelligence Online,2024-06-18T13:28:08Z,['AZ3BZ9BR'],NaN,NaN
1,1,Silent Warfare: Understanding the World of Int...,Book,https://www.nebraskapress.unl.edu/potomac-book...,NaN,https://www.zotero.org/groups/intelligence_bib...,2002,"Abram N. Shulsky, Gary Schmitt",Potomac Books,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN,NaN
2,2,"The Secret Intelligence Service, Passport Cont...",Journal article,https://doi.org/10.1093/ehr/ceae105,The role of members of the British Secret Inte...,https://www.zotero.org/groups/intelligence_bib...,2024-06-11,Christopher Baxter,NaN,The English Historical Review,2024-06-13T13:11:39Z,['SCCGXHMZ'],10.1093/ehr/ceae105,NaN
3,3,The Intelligence Bureau [India],Book chapter,https://doi.org/10.1093/oso/9780198894612.003....,This chapter reviews one of the most enigmatic...,https://www.zotero.org/groups/intelligence_bib...,2024-03-28,Praveen Swami,Oxford University Press,NaN,2024-06-17T11:33:25Z,"['CK5MNYPQ', 'AZ3BZ9BR']",NaN,Institutional Roots of India's Security Policy
4,4,The KGB’s Black Box: Intelligence Operations a...,Journal article,https://journals.library.columbia.edu/index.ph...,The Soviet Union's intelligence operations in ...,https://www.zotero.org/groups/intelligence_bib...,2024-04-15,Dale Satre,NaN,The Columbia Journal of Asia,2024-06-17T11:23:01Z,['CZT6L9T7'],10.52214/cja.v2i2.11059,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6896,6896,The Uses and Limits of Intelligence,Book,https://doi.org/10.4324/9781351302128,NaN,https://www.zotero.org/groups/intelligence_bib...,2017,Walter Laqueur,Routledge,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN,NaN
6897,6897,"Analyzing Intelligence: Origins, Obstacles, an...",Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2008,"Roger Z. George, James B. Bruce",Georgetown University Press,NaN,2020-06-05T10:17:21Z,['HCN8YFI8'],NaN,NaN
6898,6898,Intelligence Services in the Information Age,Book,https://doi.org/10.4324/9780203479667,NaN,https://www.zotero.org/groups/intelligence_bib...,2013,Michael Herman,Routledge,NaN,2020-06-05T10:17:21Z,['HCN8YFI8'],NaN,NaN
6899,6899,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN,NaN


# Retrieving citation count and open access status

In [322]:
df_doi = pd.read_csv('all_items.csv')
if datetime.today().weekday() == 6:

    df_doi = df_doi[['Zotero link', 'DOI']].dropna()
    df_doi = df_doi.drop(df_doi[df_doi['DOI'] == ''].index)
    df_doi = df_doi.reset_index(drop=True)
    df_doi['DOI'] = df_doi['DOI'].str.replace('https://doi.org/', '')

    def fetch_article_metadata(doi):
        base_url = 'https://api.openalex.org/works/https://doi.org/'
        response = requests.get(base_url + doi)
        if response.status_code == 200:
            data = response.json()
            counts_by_year = data.get('counts_by_year', [])
            if counts_by_year:
                first_citation_year = min(entry.get('year') for entry in data['counts_by_year'])
            else:
                first_citation_year = None
            if data.get('counts_by_year'):
                last_citation_year = max(entry.get('year') for entry in data['counts_by_year'])
            else:
                last_citation_year = None
    
            article_metadata = {
                'ID': data.get('id'),
                'Citation': data.get('cited_by_count'),
                'OA status': data.get('open_access', {}).get('is_oa'),
                'Citation_list': data.get('cited_by_api_url'),
                'First_citation_year': first_citation_year,
                'Last_citation_year':last_citation_year,
                'Publication_year':data.get('publication_year'),
                'OA_link':data.get('open_access', {}).get('oa_url')
            }
            return article_metadata
        else:
            return {'ID': None, 'Citation': None, 'OA status': None, 'First_citation_year': None, 'Last_citation_year':None, 'Publication_year' : None, 'OA_link':None}
    df_doi['ID'] = None
    df_doi['Citation'] = None
    df_doi['OA status'] = None
    df_doi['Citation_list'] = None
    df_doi['First_citation_year'] = None
    df_doi['Last_citation_year']=None
    df_doi['Publication_year'] = None
    df_doi['OA_link'] = None

    # Iterate over each row in the DataFrame
    for index, row in df_doi.iterrows():
        doi = row['DOI']
        article_metadata = fetch_article_metadata(doi)
        if article_metadata:
            # Update DataFrame with fetched information
            df_doi.at[index, 'ID'] = article_metadata['ID']
            df_doi.at[index, 'Citation'] = article_metadata['Citation']
            df_doi.at[index, 'OA status'] = article_metadata['OA status']
            df_doi.at[index, 'First_citation_year'] = article_metadata['First_citation_year']
            df_doi.at[index, 'Last_citation_year'] = article_metadata['Last_citation_year']
            if 'Citation_list' in article_metadata:
                df_doi.at[index, 'Citation_list'] = article_metadata['Citation_list']
            else:
                # Set to None if key is not found
                df_doi.at[index, 'Citation_list'] = None
            df_doi.at[index, 'Publication_year'] = article_metadata['Publication_year']
            if 'OA_link' in article_metadata:
                df_doi.at[index, 'OA_link'] = article_metadata['OA_link']
            else:
                # Set to None if key is not found
                df_doi.at[index, 'OA_link'] = None

    # Calculate the difference between First_citation_year and Publication_year
    df_doi['Year_difference'] = df_doi['First_citation_year'] - df_doi['Publication_year']
    df_doi.to_csv('citations.csv')
else:
    print('This part will be refreshed every Wednesday!')
df_doi

This part will be refreshed every Wednesday!


,Unnamed: 0,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,Date added,Col key,DOI,Book_title
0,0,France : French intelligence services face pos...,Newspaper article,https://www.intelligenceonline.com/government-...,The looming possibility of having a president ...,https://www.zotero.org/groups/intelligence_bib...,2024-06-18T04:00:00Z,Intelligence Online,NaN,Intelligence Online,2024-06-18T13:28:08Z,['AZ3BZ9BR'],NaN,NaN
1,1,Silent Warfare: Understanding the World of Int...,Book,https://www.nebraskapress.unl.edu/potomac-book...,NaN,https://www.zotero.org/groups/intelligence_bib...,2002,"Abram N. Shulsky, Gary Schmitt",Potomac Books,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN,NaN
2,2,"The Secret Intelligence Service, Passport Cont...",Journal article,https://doi.org/10.1093/ehr/ceae105,The role of members of the British Secret Inte...,https://www.zotero.org/groups/intelligence_bib...,2024-06-11,Christopher Baxter,NaN,The English Historical Review,2024-06-13T13:11:39Z,['SCCGXHMZ'],10.1093/ehr/ceae105,NaN
3,3,The Intelligence Bureau [India],Book chapter,https://doi.org/10.1093/oso/9780198894612.003....,This chapter reviews one of the most enigmatic...,https://www.zotero.org/groups/intelligence_bib...,2024-03-28,Praveen Swami,Oxford University Press,NaN,2024-06-17T11:33:25Z,"['CK5MNYPQ', 'AZ3BZ9BR']",NaN,Institutional Roots of India's Security Policy
4,4,The KGB’s Black Box: Intelligence Operations a...,Journal article,https://journals.library.columbia.edu/index.ph...,The Soviet Union's intelligence operations in ...,https://www.zotero.org/groups/intelligence_bib...,2024-04-15,Dale Satre,NaN,The Columbia Journal of Asia,2024-06-17T11:23:01Z,['CZT6L9T7'],10.52214/cja.v2i2.11059,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6896,6896,The Uses and Limits of Intelligence,Book,https://doi.org/10.4324/9781351302128,NaN,https://www.zotero.org/groups/intelligence_bib...,2017,Walter Laqueur,Routledge,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN,NaN
6897,6897,"Analyzing Intelligence: Origins, Obstacles, an...",Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2008,"Roger Z. George, James B. Bruce",Georgetown University Press,NaN,2020-06-05T10:17:21Z,['HCN8YFI8'],NaN,NaN
6898,6898,Intelligence Services in the Information Age,Book,https://doi.org/10.4324/9780203479667,NaN,https://www.zotero.org/groups/intelligence_bib...,2013,Michael Herman,Routledge,NaN,2020-06-05T10:17:21Z,['HCN8YFI8'],NaN,NaN
6899,6899,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN,NaN


In [323]:
# ## Remove after the first successful retrieval 2024-02-18

# df_doi = pd.read_csv('all_items.csv')
# if datetime.today().weekday() == 3:

#     df_doi = df_doi[['Zotero link', 'DOI']].dropna()
#     df_doi = df_doi.drop(df_doi[df_doi['DOI'] == ''].index)
#     df_doi = df_doi.reset_index(drop=True)
#     df_doi['DOI'] = df_doi['DOI'].str.replace('https://doi.org/', '')

#     def fetch_article_metadata(doi):
#         base_url = 'https://api.openalex.org/works/https://doi.org/'
#         response = requests.get(base_url + doi)
#         if response.status_code == 200:
#             data = response.json()
#             counts_by_year = data.get('counts_by_year', [])
#             if counts_by_year:
#                 first_citation_year = min(entry.get('year') for entry in data['counts_by_year'])
#             else:
#                 first_citation_year = None
#             article_metadata = {
#                 'ID': data.get('id'),
#                 'Citation': data.get('cited_by_count'),
#                 'OA status': data.get('open_access', {}).get('is_oa'),
#                 'Citation_list': data.get('cited_by_api_url'),
#                 'First_citation_year': first_citation_year,
#                 'Publication_year':data.get('publication_year')
#             }
#             return article_metadata
#         else:
#             return {'ID': None, 'Citation': None, 'OA status': None, 'First_citation_year': None, 'Publication_year' : None}
#     df_doi['ID'] = None
#     df_doi['Citation'] = None
#     df_doi['OA status'] = None
#     df_doi['Citation_list'] = None
#     df_doi['First_citation_year'] = None
#     df_doi['Publication_year'] = None

#     # Iterate over each row in the DataFrame
#     for index, row in df_doi.iterrows():
#         doi = row['DOI']
#         article_metadata = fetch_article_metadata(doi)
#         if article_metadata:
#             # Update DataFrame with fetched information
#             df_doi.at[index, 'ID'] = article_metadata['ID']
#             df_doi.at[index, 'Citation'] = article_metadata['Citation']
#             df_doi.at[index, 'OA status'] = article_metadata['OA status']
#             df_doi.at[index, 'First_citation_year'] = article_metadata['First_citation_year']
#             if 'Citation_list' in article_metadata:
#                 df_doi.at[index, 'Citation_list'] = article_metadata['Citation_list']
#             else:
#                 # Set to None if key is not found
#                 df_doi.at[index, 'Citation_list'] = None
#             df_doi.at[index, 'Publication_year'] = article_metadata['Publication_year']

#     # Calculate the difference between First_citation_year and Publication_year
#     df_doi['Year_difference'] = df_doi['First_citation_year'] - df_doi['Publication_year']
#     df_doi.to_csv('citations.csv')
# else:
#     print('This part will be refreshed every Wednesday!')
# df_doi

In [324]:
# ## Retrieving citation and open access status without first ciation year
# df_doi = pd.read_csv('all_items.csv')
# # df_doi = df_doi.head(10)
# if datetime.today().weekday() == 5:

#     df_doi = df_doi[['Zotero link', 'DOI']].dropna()
#     df_doi = df_doi.drop(df_doi[df_doi['DOI']==''].index)
#     df_doi = df_doi.reset_index(drop=True)
#     df_doi['DOI'] = df_doi['DOI'].str.replace('https://doi.org/', '')
#     df_doi

#     def fetch_article_metadata(doi):
#         base_url = 'https://api.openalex.org/works/https://doi.org/'
#         response = requests.get(base_url + doi)
#         if response.status_code == 200:
#             data = response.json()
#             article_metadata = {
#                 'ID': data.get('id'),
#                 'Citation': data.get('cited_by_count'),
#                 'OA status': data.get('open_access', {}).get('is_oa'),
#                 'Citation_list':data.get('cited_by_api_url')
#             }
#             return article_metadata
#         else:
#             return {'ID': None, 'Citation': None, 'OA status': None}

#     df_doi['ID'] = None
#     df_doi['Citation'] = None
#     df_doi['OA status'] = None
#     df_doi['Citation_list'] = None

#     # Iterate over each row in the DataFrame
#     for index, row in df_doi.iterrows():
#         doi = row['DOI']
#         article_metadata = fetch_article_metadata(doi)
#         if article_metadata:
#             # Update DataFrame with fetched information
#             df_doi.at[index, 'ID'] = article_metadata['ID']
#             df_doi.at[index, 'Citation'] = article_metadata['Citation']
#             df_doi.at[index, 'OA status'] = article_metadata['OA status']
#             if 'Citation_list' in article_metadata:
#                 df_doi.at[index, 'Citation_list'] = article_metadata['Citation_list']
#             else:
#                 # Set to None if key is not found
#                 df_doi.at[index, 'Citation_list'] = None
#     df_doi
#     df_doi.to_csv('citations.csv')
# else:
#     print('This part will be refreshed every Wednesday!')

In [325]:
df_doi

,Unnamed: 0,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,Date added,Col key,DOI,Book_title
0,0,France : French intelligence services face pos...,Newspaper article,https://www.intelligenceonline.com/government-...,The looming possibility of having a president ...,https://www.zotero.org/groups/intelligence_bib...,2024-06-18T04:00:00Z,Intelligence Online,NaN,Intelligence Online,2024-06-18T13:28:08Z,['AZ3BZ9BR'],NaN,NaN
1,1,Silent Warfare: Understanding the World of Int...,Book,https://www.nebraskapress.unl.edu/potomac-book...,NaN,https://www.zotero.org/groups/intelligence_bib...,2002,"Abram N. Shulsky, Gary Schmitt",Potomac Books,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN,NaN
2,2,"The Secret Intelligence Service, Passport Cont...",Journal article,https://doi.org/10.1093/ehr/ceae105,The role of members of the British Secret Inte...,https://www.zotero.org/groups/intelligence_bib...,2024-06-11,Christopher Baxter,NaN,The English Historical Review,2024-06-13T13:11:39Z,['SCCGXHMZ'],10.1093/ehr/ceae105,NaN
3,3,The Intelligence Bureau [India],Book chapter,https://doi.org/10.1093/oso/9780198894612.003....,This chapter reviews one of the most enigmatic...,https://www.zotero.org/groups/intelligence_bib...,2024-03-28,Praveen Swami,Oxford University Press,NaN,2024-06-17T11:33:25Z,"['CK5MNYPQ', 'AZ3BZ9BR']",NaN,Institutional Roots of India's Security Policy
4,4,The KGB’s Black Box: Intelligence Operations a...,Journal article,https://journals.library.columbia.edu/index.ph...,The Soviet Union's intelligence operations in ...,https://www.zotero.org/groups/intelligence_bib...,2024-04-15,Dale Satre,NaN,The Columbia Journal of Asia,2024-06-17T11:23:01Z,['CZT6L9T7'],10.52214/cja.v2i2.11059,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6896,6896,The Uses and Limits of Intelligence,Book,https://doi.org/10.4324/9781351302128,NaN,https://www.zotero.org/groups/intelligence_bib...,2017,Walter Laqueur,Routledge,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN,NaN
6897,6897,"Analyzing Intelligence: Origins, Obstacles, an...",Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2008,"Roger Z. George, James B. Bruce",Georgetown University Press,NaN,2020-06-05T10:17:21Z,['HCN8YFI8'],NaN,NaN
6898,6898,Intelligence Services in the Information Age,Book,https://doi.org/10.4324/9780203479667,NaN,https://www.zotero.org/groups/intelligence_bib...,2013,Michael Herman,Routledge,NaN,2020-06-05T10:17:21Z,['HCN8YFI8'],NaN,NaN
6899,6899,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,2020-06-05T10:17:21Z,"['HCN8YFI8', 'DVEM4H4W']",NaN,NaN


In [326]:
df_doi_2 = pd.read_csv('citations.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_2

,Zotero link,DOI,ID,Citation,OA status,Citation_list,First_citation_year,Last_citation_year,Publication_year,OA_link,Year_difference
0,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2024.2350423,https://openalex.org/W4399625728,0.0,False,https://api.openalex.org/works?filter=cites:W4...,NaN,NaN,2024.0,NaN,NaN
1,https://www.zotero.org/groups/intelligence_bib...,10.1177/13691481241258108,https://openalex.org/W4399706853,0.0,False,https://api.openalex.org/works?filter=cites:W4...,NaN,NaN,2024.0,NaN,NaN
2,https://www.zotero.org/groups/intelligence_bib...,10.1093/ehr/ceae105,https://openalex.org/W4399512677,0.0,False,https://api.openalex.org/works?filter=cites:W4...,NaN,NaN,2024.0,NaN,NaN
3,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2024.2340955,https://openalex.org/W4399512774,0.0,True,https://api.openalex.org/works?filter=cites:W4...,NaN,NaN,2024.0,https://doi.org/10.1080/08850607.2024.2340955,NaN
4,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2024.2343258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3602,https://www.zotero.org/groups/intelligence_bib...,10.1177/1755088217712069,https://openalex.org/W2612406272,6.0,True,https://api.openalex.org/works?filter=cites:W2...,2019.0,2024.0,2017.0,https://journals.sagepub.com/doi/pdf/10.1177/1...,2.0
3603,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2019.1660951,https://openalex.org/W2996219269,5.0,False,https://api.openalex.org/works?filter=cites:W2...,2021.0,2023.0,2019.0,NaN,2.0
3604,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2012.705186,https://openalex.org/W2033999438,26.0,False,https://api.openalex.org/works?filter=cites:W2...,2013.0,2022.0,2013.0,NaN,0.0
3605,https://www.zotero.org/groups/intelligence_bib...,10.1080/08850607.2020.1748993,https://openalex.org/W3022421216,1.0,False,https://api.openalex.org/works?filter=cites:W3...,2024.0,2024.0,2020.0,NaN,4.0


In [327]:
df_1 = pd.read_csv('all_items.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_1 = pd.read_csv('citations.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_1 = df_doi_1.drop(columns=['DOI'])
df_1 = pd.merge(df_1, df_doi_1, on='Zotero link', how='left')
df_1.to_csv('all_items.csv')

In [328]:
df_2 = pd.read_csv('all_items_duplicated.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_2 = pd.read_csv('citations.csv', usecols=lambda column: column != 'Unnamed: 0')
df_doi_2 = df_doi_2.drop(columns=['DOI'])
df_2 = pd.merge(df_2, df_doi_2, on='Zotero link', how='left')
df_2.to_csv('all_items_duplicated.csv')

In [329]:
df_2 = pd.read_csv('all_items_duplicated.csv')
df_2

,Unnamed: 0,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,...,Collection_Link,ID,Citation,OA status,Citation_list,First_citation_year,Last_citation_year,Publication_year,OA_link,Year_difference
0,0,France : French intelligence services face pos...,Newspaper article,https://www.intelligenceonline.com/government-...,The looming possibility of having a president ...,https://www.zotero.org/groups/intelligence_bib...,2024-06-18T04:00:00Z,Intelligence Online,NaN,Intelligence Online,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Silent Warfare: Understanding the World of Int...,Book,https://www.nebraskapress.unl.edu/potomac-book...,NaN,https://www.zotero.org/groups/intelligence_bib...,2002,"Abram N. Shulsky, Gary Schmitt",Potomac Books,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Silent Warfare: Understanding the World of Int...,Book,https://www.nebraskapress.unl.edu/potomac-book...,NaN,https://www.zotero.org/groups/intelligence_bib...,2002,"Abram N. Shulsky, Gary Schmitt",Potomac Books,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"The Secret Intelligence Service, Passport Cont...",Journal article,https://doi.org/10.1093/ehr/ceae105,The role of members of the British Secret Inte...,https://www.zotero.org/groups/intelligence_bib...,2024-06-11,Christopher Baxter,NaN,The English Historical Review,...,https://www.zotero.org/groups/intelligence_bib...,https://openalex.org/W4399512677,0.0,False,https://api.openalex.org/works?filter=cites:W4...,NaN,NaN,2024.0,NaN,NaN
4,4,The Intelligence Bureau [India],Book chapter,https://doi.org/10.1093/oso/9780198894612.003....,This chapter reviews one of the most enigmatic...,https://www.zotero.org/groups/intelligence_bib...,2024-03-28,Praveen Swami,Oxford University Press,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317,10317,Intelligence Services in the Information Age,Book,https://doi.org/10.4324/9780203479667,NaN,https://www.zotero.org/groups/intelligence_bib...,2013,Michael Herman,Routledge,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10318,10318,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10319,10319,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10320,10320,Intelligence and Government in Britain and the...,Book,NaN,NaN,https://www.zotero.org/groups/intelligence_bib...,2012,Philip H.J. Davies,Praeger,NaN,...,https://www.zotero.org/groups/intelligence_bib...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [330]:
ttt = df_2[df_2['DOI']=='10.1080/08850600600829882']
ttt

,Unnamed: 0,Title,Publication type,Link to publication,Abstract,Zotero link,Date published,FirstName2,Publisher,Journal,...,Collection_Link,ID,Citation,OA status,Citation_list,First_citation_year,Last_citation_year,Publication_year,OA_link,Year_difference
7618,7618,Modeling an Intelligence Analysis Profession o...,Journal article,https://doi.org/10.1080/08850600600829882,NaN,https://www.zotero.org/groups/intelligence_bib...,2006-12-01,"Stephen Marrin, Jonathan D. Clemente",NaN,Intl Journal of Intelligence and Counterintell...,...,https://www.zotero.org/groups/intelligence_bib...,https://openalex.org/W2074061346,16.0,True,https://api.openalex.org/works?filter=cites:W2...,2012.0,2023.0,2006.0,http://www.stuart-hall.com/ftp/IJIC--Vol19No4-...,6.0


In [331]:
# df_zotero_id = df_2[['Zotero link']].copy()
# df_zotero_id['Zotero link'] = df_zotero_id['Zotero link'].str.replace(
#     'https://www.zotero.org/groups/intelligence_bibliography/items/', ''
# )
# df_zotero_id = df_zotero_id.drop_duplicates(subset='Zotero link').reset_index(drop=True)
# df_zotero_id = df_zotero_id.rename(columns={'Zotero link':'zotero_item_key'})
# # df_zotero_id = df_zotero_id.head(100)
# df_zotero_id

In [349]:
df_zotero_id = pd.read_csv('https://raw.githubusercontent.com/yusufaliozkan/zotero-intelligence-bibliography/main/zotero_citation_format.csv')
df_all = pd.read_csv('all_items.csv')
df_all = df_all[['Zotero link']]
df_all['zotero_item_key'] = df_all['Zotero link'].str.replace('https://www.zotero.org/groups/intelligence_bibliography/items/', '')
df_all = df_all.drop_duplicates()
df_not_zotero_id = df_all[~df_all['zotero_item_key'].isin(df_zotero_id['zotero_item_key'])]
df_not_zotero_id = df_not_zotero_id[['zotero_item_key']].reset_index(drop=True)
df_not_zotero_id

C:\Users\yaozk\AppData\Local\Temp\ipykernel_17372\1839613883.py:4: FutureWarning:

The default value of regex will change from True to False in a future version.



,zotero_item_key
0,L3ALJXEQ
1,RYINFHY6
2,E8Y9Q4AD
3,W9T2973S
4,U7LEZPWP
5,6JEBZY2F
6,XI3JE5AS
7,7TCXXGIV
8,3TZ56WTC
9,9A2URZQ4


In [350]:
user_id = '2514686'

# Base URL for Zotero API
base_url = 'https://api.zotero.org'

# Initialize an empty string to accumulate bibliographies
all_bibliographies = ""

# List to store bibliographies
bibliographies = []

# Iterate through each item key in the DataFrame
for item_key in df_not_zotero_id['zotero_item_key']:
    # Endpoint to get item bibliography
    endpoint = f'/groups/{user_id}/items/{item_key}'

    # Parameters for the request
    params = {
        'format': 'bib',
        'linkwrap': 1
    }

    # Make GET request to Zotero API
    response = requests.get(base_url + endpoint, params=params)

    # Check if request was successful
    if response.status_code == 200:
        bibliography = response.text.strip()  # Strip any leading/trailing whitespace
        bibliographies.append(bibliography)
        all_bibliographies += f'<p>{bibliography}</p><br><br>'  # Append bibliography with two newlines for separation
    else:
        error_message = f'Error fetching bibliography for item {item_key}: Status Code {response.status_code}'
        bibliographies.append(error_message)
        all_bibliographies += f'<p>{error_message}</p><br><br>'

# Add bibliographies to the original DataFrame
df_not_zotero_id['bibliography'] = bibliography
df_not_zotero_id

,zotero_item_key,bibliography
0,L3ALJXEQ,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
1,RYINFHY6,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
2,E8Y9Q4AD,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
3,W9T2973S,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
4,U7LEZPWP,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
5,6JEBZY2F,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
6,XI3JE5AS,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
7,7TCXXGIV,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
8,3TZ56WTC,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
9,9A2URZQ4,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."


In [351]:
bibliographies

['<?xml version="1.0"?>\n<div class="csl-bib-body" style="line-height: 1.35; padding-left: 1em; text-indent:-1em;">\n  <div class="csl-entry">Intelligence Online. &#x201C;France&#x202F;: French Intelligence Services Face Possible Political Cohabitation Test.&#x201D; <i>Intelligence Online</i>, June 18, 2024. <a href="https://www.intelligenceonline.com/government-intelligence/2024/06/17/french-intelligence-services-face-possible-political-cohabitation-test,110248229-eve">https://www.intelligenceonline.com/government-intelligence/2024/06/17/french-intelligence-services-face-possible-political-cohabitation-test,110248229-eve</a>.</div>\n</div>',
 '<?xml version="1.0"?>\n<div class="csl-bib-body" style="line-height: 1.35; padding-left: 1em; text-indent:-1em;">\n  <div class="csl-entry">Swami, Praveen. &#x201C;The Intelligence Bureau [India].&#x201D; In <i>Institutional Roots of India&#x2019;s Security Policy</i>, edited by Milan Vaishnav, 0. Oxford: Oxford University Press, 2024. <a href="

In [357]:
df_zotero_id = pd.read_csv('https://raw.githubusercontent.com/yusufaliozkan/zotero-intelligence-bibliography/main/zotero_citation_format.csv', index_col=False)
df_zotero_id = df_zotero_id.drop(columns={'Unnamed: 0'})
df_zotero_id = pd.concat([df_zotero_id, df_not_zotero_id]).reset_index(drop=True)
df_zotero_id

,zotero_item_key,bibliography
0,MM95S3LQ,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
1,AN5YI8EM,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
2,UYS2PIFQ,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
3,55Z2ZJDM,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
4,GMC4REMY,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
...,...,...
6896,SX23WPGA,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
6897,E9NJ54MR,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
6898,MRAC99ZE,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."
6899,RM26JRH6,"<?xml version=""1.0""?>\n<div class=""csl-bib-bod..."


In [358]:
df_zotero_id.to_csv('zotero_citation_format.csv')